# 규제 선형 모델

- 다항식이 복잡해지면서 회귀계수 값이 크게 설정되고 이는 과대적합을 유발함
    - 과대적합이 일어나면 훈련데이터에서는 높은 성능을 보이지만 테스트데이터에서는 낮은 성능을 보임
- 선형 회귀 모델의 목적은 비용 함수를 최소화 하는 것
- 이때, 규제 선형 모델은 비용함수를 최고화하는 데에 패널티를 적용해 회귀계수의 크기를 제한함

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split

boston_df = pd.read_csv('./data/boston_housing_train.csv')

X = boston_df.drop('MEDV', axis=1)
y = boston_df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score

def evaluate_regression(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"MSE: {mse} | RMSE: {rmse} | MAE: {mae} | R2score: {r2}")
    return

### Ridge 회귀

- L2 규제를 적용한 회귀 모델
- 회귀 계수들(가중치, 모델 파라미터)의 제곱합에 alpha 패널티를 곱해 비용 산정에 반영
    - alpha 값이 커지면 회귀 계수가 작아짐


In [6]:
from sklearn.linear_model import Ridge

reg_l2 = Ridge()
reg_l2.fit(X_train, y_train)
print(reg_l2.score(X_train, y_train), reg_l2.score(X_test, y_test))
evaluate_regression(y_test, reg_l2.predict(X_test))

0.7461161787884155 0.678974832784608
MSE: 22.480475501233855 | RMSE: 4.741357980709098 | MAE: 3.050375126006179 | R2score: 0.678974832784608


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

reg_l2_model = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False, degree=2))
    , ('reg_l2', Ridge(alpha=100))
])

reg_l2_model.fit(X_train, y_train)
print(reg_l2_model.score(X_train, y_train), reg_l2_model.score(X_test, y_test))

evaluate_regression(y_train, reg_l2_model.predict(X_train))
evaluate_regression(y_test, reg_l2_model.predict(X_test))

0.922557959447853 0.796056183892697
MSE: 6.867694250081351 | RMSE: 2.620628598272054 | MAE: 1.9687946082912124 | R2score: 0.922557959447853
MSE: 14.281602907949667 | RMSE: 3.7791008067991076 | MAE: 2.4589907450590327 | R2score: 0.796056183892697


### Lasso 회귀

- L1 규제를 적용한 선형 회귀 모델
- 회귀계수 절대값의 합을 alpha 값과 곱해 규제 정도 지정
    - alpha 값이 커지면 규제 정도가 커짐
- 회귀계수를 0까지 줄일 수 있음 == 해당 특성을 아예 반영하지 않도록 함 == 특성 선택 효과

In [20]:
from sklearn.linear_model import Lasso

# reg_l1 = Lasso(alpha=1) # 기본값
# reg_l1 = Lasso(alpha=1000) # 1000쯤 되면 모든 가중치가 0이 됨
reg_l1 = Lasso()
reg_l1.fit(X_train, y_train)
print(reg_l1.score(X_train, y_train), reg_l1.score(X_test, y_test))
print(reg_l1.coef_)

0.6948040743556285 0.6516957380017043
[-0.0838981   0.02646051 -0.          0.          0.          1.54544951
  0.01345772 -0.58282853  0.20738089 -0.01121302 -0.70500625  0.01198848
 -0.75783702]


### ElasticNet 회귀

- L1, L2 규제를 함께 적용하는 회귀 모델
- 하이퍼 파라미터
    - alpha : 규제 정도 (L1규제 + L2규제)
    - l1_ratio : L1 규제의 비율 (L2 규제의 비율 == 1 -l1_ratio)

In [21]:
from sklearn.linear_model import ElasticNet

reg_ela = ElasticNet(alpha=1, l1_ratio=0.5) # 기본값

reg_ela.fit(X_train, y_train)
print(reg_ela.score(X_train, y_train), reg_ela.score(X_test, y_test))
print(reg_ela.coef_)

0.6910880311369155 0.6559413789855975
[-0.10270885  0.03414157 -0.00689058  0.          0.          1.14428334
  0.01442808 -0.70930241  0.26472536 -0.01347162 -0.74428292  0.01200929
 -0.78282769]
